# Implementation of Retina Network.

(Putting it all together)

In [1]:
from importlib.util import find_spec
if find_spec("model") is None:
    import sys
    sys.path.append('..')

In [2]:
from typing import Optional

In [3]:
from base import BaseModel
from model.backbone.resnet import ResNet50
from model.backbone.retina_meta import RetinaNetFPN50, RetinaNetHead
from model.anchor_generator import AnchorBoxGenerator

In [4]:
class ResnetModel(BaseModel):
    def __init__(self, num_classes: Optional[int] = 80):
        super().__init__()

        sizes = [32.0, 64.0, 128.0, 256.0, 512.0]
        aspect_ratios = [0.5, 1.0, 2.0]
        scales = [1.0, 2 ** (1 / 3), 2 ** (2 / 3)]
        strides = [2, 2, 2, 2, 2]

        self.base = ResNet50()
        self.backbone = RetinaNetFPN50()
        self.head = RetinaNetHead(num_classes)
        self.anchors = AnchorBoxGenerator(sizes, aspect_ratios, strides, scales)

    def forward(self, x):
        _, C3, C4, C5 = self.base(x)
        P3, P4, P5, P6, P7 = self.backbone(C3, C4, C5)

        return self.head(P3, P4, P5, P6, P7)

In [5]:
model = ResnetModel()

In [7]:
model

ResnetModel(
  (base): ResNet50(
    (stem): FastStem(
      (conv1_1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2))
      (conv1_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv1_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn1_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn1_3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (res_block2_1): BottleNeckBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
 